# Playing with Boxes

In this notebook, we will:
1. Create a tensor representation of a box
2. Represent specific boxes using your tensor representation
3. Create a function which calculates the intersection of two boxes
3. Create a differentiable function to compute "soft" volumes of boxes
4. Train one box to contain another

This notebook is intended to be self-contained, but you may find it beneficial to consult Section 3.1 to 3.4 in [Representing Joint Hierarchies with Box Embeddings](https://openreview.net/pdf?id=J246NSqR_l).

## 1. Create a box parameterization

A "box" is a product of intervals in $\mathbb R^n$, i.e.

$$X = \prod_{i=1}^d [x_{m,i}, x_{M,i}], \quad \text{where} \quad x_{M,i} \ge x_{m,i} \quad \text{ for all } \quad i.$$

However you would like, create a way of storing parameters which represent a box. Crucially, your parameterization should conform to the requirement above, namely any setting of your parameterization should represent a box that has positive side-lengths in each dimension, and you should be able to represent any box in $\mathbb R^n$. Furthermore, you should be able to return the min and max coordinates (i.e. the $x_{m,i}, x_{M, i}$ above) for your box.

In [12]:
# Box parameterization


## 2. Represent Specific Boxes
Using your chosen parameterization, represent a box `x` in $\mathbb R^{100}$ which has side-lengths $2$ centered at $(-,1,\ldots, -1)$. Create another box `y` with min-coordinate $(1, \frac 1 2, \frac 1 3, \ldots, \frac 1 {100})$, and max-coordinate $(3 - \frac 1 {100}, 3 - \frac 1 {99}, \ldots, 2)$.


In [ ]:
# Represent a specific box

## 3. Create an Intersection Function

The box intersection operation takes two boxes as input and returns their intersection. In one-dimension, this is as follows:

$$\text{Int}([x_m, x_M], [y_m, y_M]) = (\max(x_m, y_m), \min(x_M, y_M)).$$

Generalize this to $n$-dimensions, and write an intersection function which operates on your box parameterization.

Take the intersection of the two boxes `x` and `y` you created in step 2, and display the min and max coordinates of the intersection.

In [ ]:
# Intersection Function

## 4. Create a "Soft" Volume Function
The "softplus volume" of a box is defined as

$$\text{SoftVol}[X] = \prod_{i=1}^d \log(1 + \exp(x_{M,i} - x_{m,i})).$$

Create this volume function, and attempt to use it to determine which of the boxes `x` and `y` from part (2) are larger. What issues do you encounter? Can you mitigate these issues?

In [ ]:
# Soft Volume Function

## 5. Training Boxes
We say box $X$ contains box $Y$ if, for each dimension $i$, we have
$$x_{m,i} < y_{m,i} < y_{M,i} < x_{M,i}.$$

Freezing the coordinates of box `x` from earlier, use gradient-descent to train box `y` such that it is contained in `x`.

**Hint:** Visualize typical boxes in 2-dimensions, and note that if a box $Y$ were contained in $X$ then $\text{Vol}(\text{Int}(X, Y)) = \text{Vol}(Y)$.

In [ ]:
# Training boxes

